In [1]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
directory = "4_01/preds/"
pred_files = os.listdir(directory)

In [3]:
for n, pred_file in enumerate(pred_files):
    if n == 0:
        d = pd.read_csv(directory + pred_file, sep = "\t").drop(columns = ["foto_mes"]).rename(columns = {"prob":pred_file})
    else:
        d2 = pd.read_csv(directory + pred_file, sep = "\t").drop(columns = ["foto_mes"]).rename(columns = {"prob":pred_file})
        d = pd.merge(d,d2, on = "numero_de_cliente", how = "outer")

In [4]:
d = d.transpose()
d.columns = d.loc["numero_de_cliente"].astype(int).astype(str)
d = d.drop("numero_de_cliente")

In [5]:
d_ordered =  d.rank(ascending = False,axis = 1).reindex(d.max().sort_values(ascending = False).index, axis=1)

In [13]:
d_ordered.min().value_counts(sort = False)

61848.0    1
39574.0    1
33543.0    1
2479.0     4
21401.0    1
          ..
95911.0    1
1174.0     1
61597.0    3
95588.0    2
57889.0    2
Length: 89539, dtype: int64

In [16]:
ordered_by = "max"
for n_envios in [4000,5000,6000,7000,8000,9000,10000]:
    nombre_entrega = f"401_model1&2_{ordered_by}_rank_{n_envios}e.csv"
    d_entrega  = pd.DataFrame({"numero_de_cliente" : d_ordered.columns, f"{ordered_by}_rank":d_ordered.max().values})
    d_entrega["Predicted"] = 0
    d_entrega.loc[0:n_envios-1, "Predicted"] = 1
    d_entrega.drop(f"{ordered_by}_rank", axis = 1, inplace = True)
    d_entrega.to_csv(nombre_entrega, index = None)